## GPU data parallelism
* Worker = is a whole computer with CPU and all (node)
* device or accelerator is a GPU or TPU
* XLA_GPU = Accelerated Linear Algebra

In [1]:
import tensorflow as tf
import multiprocessing as mp
import os, sys
import inspect
import numpy as np
import matplotlib.pyplot as plt
import threading
import queue
import glob

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe())))
#parentdir = os.path.dirname(currentdir)
sys.path.insert(0, currentdir)
%load_ext autoreload
%autoreload 2
import gpu_wit as gw

In [2]:
promath_path = '/media/hd1/promath/'
inference_path = '/media/hd1/glossary/NN.v1/'
year_dir = 'math11'
set1 = set([os.path.split(n)[-1].split('.')[0] 
            for n in glob.glob(os.path.join(promath_path, year_dir) + '/*.tar.gz')])
set2 = set([os.path.split(n)[-1].split('.')[0] 
            for n in glob.glob(os.path.join(inference_path, year_dir) + '/*.xml.gz')])
set2.pop()
set1.difference(set2)

NameError: name 'glob' is not defined

In [3]:
cfg = {'hola': 1, 'adios': 2, 3:"macizo"}
cfg.update({'culo': 4, 'adios': 4})
cfg

{'hola': 1, 'adios': 4, 3: 'macizo', 'culo': 4}

In [12]:
def dir_diff(promath_path, inference_path, year_dir):
    '''
promath_path ex. '/media/hd1/promath/' 
                 expected file format .tar.gz
inference_path ex. '/media/hd1/glossary/NN.v1/' 
                 expected formar .xml.gz
year_dir: either a string "math91" or a list of strings ["math01", "math02", ]
    '''
    if isinstance(year_dir, str):
        year_dir = [year_dir, ]
        
    # promath and inference path lists
    pp_lst = []
    pi_lst = []
    for year in year_dir:
        pp_lst += [os.path.join(year, os.path.split(n)[-1].split('.')[0])
                for n in glob.glob(os.path.join(promath_path, year) + '/*.tar.gz')]
        pi_lst += [os.path.join(year, os.path.split(n)[-1].split('.')[0])
                for n in glob.glob(os.path.join(inference_path, year) + '/*.xml.gz')]
        # output looks like ['math11/1106_003', '']
        
    pp_set = set(pp_lst)
    pi_set = set(pi_lst)
    return pp_set.difference(pi_set)
    
def check_if_done(promath_path, mined_path):
    '''
    Inputs:
        promath_path: path to data file ex. $PROJECT/promath/math11/1101_001.tar.gz
        mined_path:   dir of already mined files. ex. $PROJECT/with_mp/
    '''
    pass

dir_diff(promath_path, inference_path, ['math' + repr(y) for y in range(91, 100)])

set()

In [118]:
#c.release()
#c.acquire()
with c._cond:
    print(c._cond)
    if not c._value:
        print('not cvalue ', c._value)
    else:
        print('yes cvalue ', c._value)

<Condition(<locked _thread.lock object at 0x7f0881d76930>, 0)>
yes cvalue  2


In [119]:
c._value

2

In [120]:
q = queue.Queue()
for i in range(3):
    q.put(i)
print(q.get())

0


In [130]:
q.qsize()
q.not_empty?

Type:        Condition
String form: <Condition(<unlocked _thread.lock object at 0x7f0881639510>, 0)>
File:        /usr/lib/python3.8/threading.py
Docstring:  
Class that implements a condition variable.

A condition variable allows one or more threads to wait until they are
notified by another thread.

If the lock argument is given and not None, it must be a Lock or RLock
object, and it is used as the underlying lock. Otherwise, a new RLock object
is created and used as the underlying lock.


In [17]:
# initialize the worker process
def init_worker():
    # get the pid for the current worker process
    pid = os.getpid()
    print(f'Worker PID: {pid}', flush=True)

def init_thread_worker():
    # get the pid for the current worker process
    #pid = os.getpid()
    pid = threading.get_ident()
    print(f'Worker PID: {pid}', flush=True)
    
with mp.pool.ThreadPool(4,initializer=init_thread_worker):
    pass

Worker PID: 139678259599104
Worker PID: 139677982770944
Worker PID: 139677974378240
Worker PID: 139677965985536


In [11]:
gpus = tf.config.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 2 Logical GPUs


In [19]:
# List devices
# If this cell is run the next cell won't work properly
gpus = tf.config.list_logical_devices('GPU')
print(gpus)
#print(gpus)
#tf.config.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')]


In [18]:
#tf.debugging.set_log_device_placement(True)
#with tf.device('/device:GPU:1'):
with tf.device('/gpu:1'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [30]:
gw.paral_threadpool_exec()

mat multi on /gpu:0
mat multi on /gpu:1
mat multi on /gpu:0
mat multi on /gpu:1
[None, None, None, None]


In [ ]:
mp.pool.ThreadPool

In [7]:
def mat_mul(gpu):
    with tf.device(gpu):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
mat_mul('/gpu:1')

In [ ]:
# Does not end
gpu_lst = [f"/gpu:{k}" for k in range(2)]
with mp.Pool(2) as pool:
    print(pool.map(mat_mul, gpu_lst))

In [7]:
def mp_wrap():
    gpu_lst = [f"/gpu:{k}" for k in range(2)]
    with mp.Pool(2) as pool:
        result = pool.apply_async(mat_mul, ('/gpu:0',)) 
        print('ya salio de aqui.')
        print(result.get(timeout=10))

mp_wrap()

ya salio de aqui.


TimeoutError: 

In [8]:
tf.config.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [19]:
with tf.device('/device:GPU:0'):       # Run nodes with GPU 0
#with tf.device(gpu.name):       # Run nodes with GPU 0
    m1 = tf.constant([[3, 5]])
    m2 = tf.constant([[2],[4]])
    product = tf.matmul(m1, m2)
product

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[26]], dtype=int32)>

In [9]:
# This works for parallel training
# I couldn't find an easy working example of inference on multiple GPUs.
mirror = tf.distribute.MirroredStrategy()
mirror.scope()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [10]:
# Another way of listing GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1522951353799500426,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17831017549081373243
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7217096090645788710
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1073741824
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13993195329145291770
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1"]

In [48]:
sec = 19*60 + 41
# approx 20 mins/ 12 files
20/12/60*229

6.361111111111112

In [46]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs=2)
parser.add_argument('--bar', nargs="+")
args = parser.parse_args('--foo a b'.split())

In [48]:
args.bar == None

True

In [51]:
assert os.path.isdir('/media/hd12'), 'hola macizo'

AssertionError: hola macizo